## To Do:
#### - Outlier Frames 200x, Re-Trainieren
#### - upload to github and clean up repo (alte Analyse bis MazeCorners behalten!)
#### - Black code formatting

# Walkthrough:

In [2]:
from Analysis.gaitanalysis import Recording2D
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [3]:
rec_list = []
for file in Path.iterdir(Path("/Users/kobel/Documents/Medizin/Doktorarbeit/Coding/Inken Analyse/Dummy Data Inken/")):
    if file.suffix == '.csv':
        csv_file = str(file)
        mp4_file = str(file.parent.joinpath(file.stem[:file.name.index('DLC_resnet50_2-1413 CatwalkMay25shuffle1_600000')] + '.mp4'))
        rec = Recording2D(filepath=csv_file, recorded_framerate=30)
        rec.run(video_filepath=mp4_file, intrinsic_camera_calibration_filepath = "calibration_inken_427img.p")
        rec.run_gait_analysis()
        rec_list.append(rec)

In [ ]:
output_path = Path("/Users/kobel/Desktop/Trash")

In [ ]:
for parameter in rec_list[0].parameters_paw_placed:
    df = pd.DataFrame(data = [])
    df['animal'] = [rec.metadata['animal'] for rec in rec_list]
    df['paradigm'] = [rec.metadata['paradigm'] for rec in rec_list]
    df[parameter] = [rec.parameters_paw_placed[parameter] for rec in rec_list]
    df.to_excel(output_path.joinpath(parameter + '_paw_placed.xlsx'), index=False)

In [ ]:
for parameter in rec_list[0].parameters_as_psth:
    df = pd.DataFrame(data = [])
    df['animal'] = [rec.metadata['animal'] for rec in rec_list]
    df['paradigm'] = [rec.metadata['paradigm'] for rec in rec_list]
    for i in range(-7, 7):
        df[i] = [rec.parameters_as_psth[parameter][i] for rec in rec_list]
    df.to_excel(output_path.joinpath(parameter + '_psth.xlsx'), index=False)

In [ ]:
for parameter in rec_list[0].parameters_as_psth:
    fig, ax = plt.subplots()
    for rec in rec_list:
        plt.plot(range(-7, 7), rec.parameters_as_psth[parameter])
    print(parameter)
    plt.show()

In [ ]:
for parameter in rec_list[0].parameters_paw_placed:
    fig, ax = plt.subplots()
    i = 0
    for rec in rec_list:
        plt.bar(i, rec.parameters_paw_placed[parameter])
        i += 1
    print(parameter)
    plt.show()